## Tecton

In [7]:
import tecton
import pandas as pd
from datetime import date, datetime, timedelta

pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)

# Retrieve the 'customer_domains_aggregated' Feature View
ws = tecton.get_workspace('prod')
fv = ws.get_feature_view('customer_domains_aggregated')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Get a range of feature data from offline store
start_time = datetime(2022, 10, 2)
end_time = start_time + timedelta(days=1)
shopper_num = 10
df_tecton = fv.get_historical_features(start_time=start_time, end_time=end_time).to_spark().toPandas()
shopperids = df_tecton.head(shopper_num)["shopper_id"].values.tolist()
df_tecton.head(shopper_num)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  shopper_id  multipurchase_count  tld_count  duplicate_slds  diff_tld_prop  \
0  100021662                    0          1               0       1.000000   
1  100028856                    0          1               0       0.500000   
2  100032762                    3          1               0       0.071429   
3  100088139                    1          1               0       0.500000   
4  100279991                    1          4               4       0.666667   
5  100325510                    2          1               0       0.090909   
6  100473207                    0          1               0       1.000000   
7  100484790                    7          2               2       0.200000   
8  100592715                    0          1               0       0.333333   
9  100725894                    0          4               0       0.190476   

   duplicate_slds_prop  average_domain_per_order  1_domain_order  \
0             1.000000                  1.000000              

## Shppperml

In [9]:
import pandas as pd
from datetime import date, datetime, timedelta

pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)

sc.install_pypi_package("lz4==3.1.10")
sc.install_pypi_package("ujson")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Package already installed for current Spark context!
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1295, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [10]:
import boto3
import hashlib
import lz4.block
import ujson

def hash_shopper_id(shopper_id, modulus):
    byte_string = shopper_id.encode("utf-8")
    hashed_hexvalue = hashlib.md5(byte_string).hexdigest()
    integer_value = int(hashed_hexvalue, 16)
    return integer_value % modulus

def get_data(date_str, shopper_id):
    s3_session = boto3.Session().resource("s3")
    s3_bucket = f"gd-gxcoreservices-prod-shopperml-data"
    prefix_value = hash_shopper_id(shopper_id, 20)
    shard_prefix = "{0:0{pad}d}".format(prefix_value, pad=2)
    key = "{}/{}/{}".format(shard_prefix, date_str, shopper_id.lower())
    obj = s3_session.Object(s3_bucket, key)
    
    response = obj.get()
    value = response["Body"].read()
    content_type = response["ContentType"]
    # Assume 'binary/octet-stream' is lz4 compressed.
    if content_type in ("binary/octet-stream", "application/x-lz4"):
        value = lz4.block.decompress(value)
    return ujson.loads(value)

#     file_content = str(lz4.block.decompress(obj.get()["Body"].read()))
#     return json.loads(file_content[2:-1].replace('\\\\"', '\\"').replace('\\\\/', '/').replace('\\\\\\\\/', '/'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def is_active(row):
    return row['isactiveflag']

def is_inactive(row):
        return not row['isactiveflag']

def get_domain(row):
    # we have a typo in 'domainame', in some data
    # some 'domain_name' are None
    domain_name = row.get('domain_name') or row.get('domainame')
    return domain_name.lower() if domain_name else None

def getDate(time):
    if time is None:
        return 0
    seconds_per_year = 86400 * 365
    delta = datetime.strptime(time[0:10],"%Y-%m-%d") - datetime(2000, 1, 1)
    return delta.total_seconds() / seconds_per_year

def extract(domainInfo):
    import re
    from pyspark.sql.functions import datediff, lower, regexp_extract, regexp_replace
    
    collector = {'shopper_id': shopperid}
    
    order_ids = set()
    active_domains = set()
    active_slds = set()
    active_tlds = set()
    active_tokens = set()
    active_total_token_len = 0
    
    order_dict = dict()  # count domains in each order
    domain_register_over_year_active = 0
    domain_register_over_year_inactive = 0

    for row in filter(is_active, domainInfo):
        domain = get_domain(row)
        order_ids.add(row['order_id'])
        active_domains.add(domain)
        
        tld_re = re.compile(r'[.].*$')
        sld = tld_re.sub('', domain)
        tld = tld_re.search(domain).group(0)
        active_tlds.add(tld)
        active_slds.add(sld)

        order_dict[row['order_id']] = 1 + order_dict.get(row['order_id'], 0)
        
        parsed_create_date = getDate(row['createdate'])
        parsed_updatedate_date = getDate(row['updatedate'])
        parsed_expirationdate_date = getDate(row['expirationdate'])

        create_to_update_year = parsed_updatedate_date - parsed_create_date
        create_to_expire_year = parsed_expirationdate_date - parsed_create_date
        
        if create_to_update_year > 1 and create_to_expire_year > 1:
            domain_register_over_year_active += 1

    for row in filter(is_inactive, domainInfo):
        domain = get_domain(row)
        if domain in active_domains:
            continue

        parsed_create_date = getDate(row['createdate'])
        parsed_updatedate_date = getDate(row['updatedate'])
        parsed_expirationdate_date = getDate(row['expirationdate'])

        create_to_update_year = parsed_updatedate_date - parsed_create_date
        create_to_expire_year = parsed_expirationdate_date - parsed_create_date
        if create_to_update_year > 1 and create_to_expire_year > 1:
            domain_register_over_year_inactive += 1

    domain_register_over_year_all = domain_register_over_year_active + domain_register_over_year_inactive

    collector['multipurchase_count'] = len(active_domains) - len(order_ids)
    collector['tld_count'] = len(active_tlds)
    collector['duplicate_slds'] = len(active_domains) - len(active_slds)

    collector['diff_tld_prop'] = float(len(active_tlds)) / len(active_domains) if len(active_domains) > 0 else 0
    collector['duplicate_slds_prop'] = float(len(active_slds)) / len(active_domains) if len(
        active_domains) > 0 else 0

    order_quant = list(order_dict.values())
    collector['average_domain_per_order'] = len(active_domains) / len(order_ids) if len(order_ids) > 0 else 0
    collector['1_domain_per_order'] = len([quant for quant in order_quant if quant == 1])
    
    collector['domain_register_over_year_active'] = domain_register_over_year_active
    collector['domain_register_over_year_inactive'] = domain_register_over_year_inactive
    collector['domain_register_over_year_all'] = domain_register_over_year_all
    
    return collector

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
domain_colums = [
    "domain_name",
    "order_id",
    "status",
    "isactiveflag",
    "billingstatus",
    "autorenewflag",
    "isproxied",
    "islocked",
    "isregistrarhold",
    "islimited",
    "issuperlocked",
    "isinternaltransfer",
    "isexpirationprotected",
    "istransferprotected",
    "issmartdomain",
    "invalidwhois",
    "fraud",
    "previousregistrarid",
    "gaining_registrar_id",
    "valuation_wholesale",
    "valuation_sale",
    "createdate",
    "expirationdate",
    "updatedate",
    "modifytime",
    "canceleddate",
    "renewaldeadline",
    "lasttransferstatusdate",
    "recordcreatedate",
    "deletedate",
    "row_num",
]

results = []
for shopperid in shopperids:
    # get_data(start_time.strftime("%Y%m%d"), shopperid).keys()
    # get_data(start_time.strftime("%Y%m%d"), shopperid)["domaininfo"][0]
    domain_data = get_data(start_time.strftime("%Y%m%d"), shopperid)["domaininfo"]
    domain_df = pd.DataFrame(domain_data, columns=domain_colums)
#     domain_df.head(5)
    domainInfo = domain_df.to_dict('records')
    res = extract(domainInfo)
    results.append(res)
df_shopperml = pd.DataFrame(results)
df_shopperml

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  shopper_id  multipurchase_count  tld_count  duplicate_slds  diff_tld_prop  \
0  100021662                    0          1               0       1.000000   
1  100028856                    0          1               0       0.500000   
2  100032762                    3          1               0       0.071429   
3  100088139                    1          1               0       0.333333   
4  100279991                    1          4               4       0.666667   
5  100325510                    2          1               0       0.090909   
6  100473207                    0          1               0       1.000000   
7  100484790                    7          2               2       0.200000   
8  100592715                    0          1               0       0.333333   
9  100725894                    0          4               0       0.181818   

   duplicate_slds_prop  average_domain_per_order  1_domain_per_order  \
0             1.000000                  1.000000          

In [17]:
get_data(start_time.strftime("%Y%m%d"), "100088139")["domaininfo"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['amybeengifts.com', 802395464.0, 8, False, 0.0, 1, 1, 1, None, 0, None, 0.0, False, False, False, None, 'N', None, None, None, None, '2015-03-06T12:54:28', '2019-03-06T12:54:28', '2017-03-10T11:06:23', '2017-04-08T11:14:24.537000', '2017-03-07T16:14:49', '2019-04-20T12:54:28', None, '2015-03-06T10:53:40.203000', '2017-03-10T11:06:23', 8], ['amybeanornaments.com', 802395464.0, 8, False, 0.0, 1, 1, 1, None, 0, None, 0.0, False, False, False, None, 'N', None, None, None, None, '2015-03-06T12:54:27', '2019-03-06T12:54:27', '2017-03-10T11:06:24', '2017-04-08T11:14:25.257000', '2017-03-07T16:14:48', '2019-04-20T12:54:27', None, '2015-03-06T10:53:42.090000', '2017-03-10T11:06:24', 7], ['amybeanhome.com', 802390088.0, 8, False, 0.0, 1, 1, 1, None, 0, None, 0.0, False, False, False, None, 'N', None, None, None, None, '2015-03-06T12:36:19', '2019-03-06T12:36:19', '2017-03-10T20:06:34', '2017-04-08T20:13:51.333000', '2017-03-10T17:01:36', '2019-04-20T12:36:19', None, '2015-03-06T10:35:38.650000

In [13]:
from pandas.util.testing import assert_frame_equal
df_tecton = df_tecton.drop('window_end_ts', axis=1).drop('_effective_timestamp', axis=1)
assert_frame_equal(df_tecton.head(shopper_num), df_shopperml)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
DataFrame.columns are different

DataFrame.columns values are different (9.09091 %)
[left]:  Index(['shopper_id', 'multipurchase_count', 'tld_count', 'duplicate_slds',
       'diff_tld_prop', 'duplicate_slds_prop', 'average_domain_per_order',
       '1_domain_order', 'domain_register_over_year_active',
       'domain_register_over_year_inactive', 'domain_register_over_year_all'],
      dtype='object')
[right]: Index(['shopper_id', 'multipurchase_count', 'tld_count', 'duplicate_slds',
       'diff_tld_prop', 'duplicate_slds_prop', 'average_domain_per_order',
       '1_domain_per_order', 'domain_register_over_year_active',
       'domain_register_over_year_inactive', 'domain_register_over_year_all'],
      dtype='object')
Traceback (most recent call last):
  File "/usr/local/lib64/python3.7/site-packages/pandas/_testing/asserters.py", line 1281, in assert_frame_equal
    obj=f"{obj}.columns",
  File "/usr/local/lib64/python3.7/site-packages/pandas/_testing/asser